# Python Imports

In [ ]:
# import the required libraries
import os
import numpy as np
import base64
import matplotlib.pyplot as plt
import tensorflow as tf

from PIL import Image
from io import BytesIO
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load Labels

In [ ]:
LABELS_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', LABELS_URL)
class_names = np.array(
    open(labels_path).read().splitlines()
)[1:]

labels = tf.constant([class_names])
print(labels)

# Create Model with Base64 Signature

### Load base model, add signature and save

In [ ]:
smodel = tf.saved_model.load("./models/base/1")

# This is the current signature, that only accepts image tensors as input
signature = smodel.signatures["serving_default"]
print(signature)

# obtain the key name of the output (typically 'dense_X')
keyOutput = next(iter(signature.structured_outputs))

@tf.function()
def my_predict(image_b64):

    # get content
    content = image_b64[0]
    
    # decode image    
    image = tf.image.decode_jpeg(content,channels=3)
    # tf.compat.v1.enable_eager_execution()
    
    # resize image
    image = tf.image.resize(image, [224, 224])
    
    # expand dimension to match signature
    image = tf.expand_dims(image, 0)
    
    # set values in -1..1 range
    image = preprocess_input(image)
        
    # execute prediction
    prediction = signature(image)

    # obtain index of maximum probability prediction
    idx = tf.argmax(prediction[keyOutput],axis=1)
    
    # obtain the label for given index
    label = tf.gather(labels, idx, batch_dims=1)

    # obtain probability from Tensor
    probability = prediction[keyOutput][0,idx[0]]
    
    # combine result in String Tensor format with [label,probability]
    result = tf.concat([label, [tf.as_string(probability)]], axis=0)
    
    # return result_tensor
    return result

# Create new signature, to read b64 images
new_signature = my_predict.get_concrete_function(
    image_b64=tf.TensorSpec(name="image_b64", shape=[1], dtype=tf.string)
)

# Save model with Base64 input signature
tf.saved_model.save(
    smodel,
    export_dir="./models/redbag/1",
    signatures=new_signature
)

# Test single image

### Load model with signature

In [ ]:
smodel = tf.saved_model.load("./models/redbag/1")

# Load model's signature
signature = smodel.signatures["serving_default"]

print(signature)

### Run Inference

<div> <img src="samples/banana.jpeg" style="width: 20%"/></div>

In [ ]:
# Image to use
testfile = "samples/banana.jpeg"

In [ ]:
# load test image
content = tf.io.read_file(testfile)

# reshape to signature's expected dimensions
content = tf.reshape(content, shape = [1])

# print(tf.print(content, summarize=3))

# obtain signature
f = smodel.signatures["serving_default"]

# run prediction
myprediction = f(image_b64=content)
print(myprediction)